# Details

## Folders

**data_specifications/**: Definitions for individual columns.

**jpx_tokyo_market_prediction/**: Files that enable the API. Expect the API to deliver all rows in under five minutes and to reserve less than 0.5 GB of memory.

Copies of data files exist in multiple folders that cover different time windows and serve different purposes.

**train_files/**: Data folder covering the main training period. There are 5 csv data in train_files

**supplemental_files/**: Data folder containing a dynamic window of supplemental training data. This will be updated with new data during the main phase of the competition in early May, early June, and roughly a week before the submissions are locked.

**example_test_files/**: Data folder covering the public test period. Intended to facilitate offline testing. Includes the same columns delivered by the API (ie no Target column). You can calculate the Target column from the Close column; it's the return from buying a stock the next day and selling the day after that. This folder also includes an example of the sample submission file that will be delivered by the API.

## Files

**stock_prices.csv**: The core file of interest. Includes the daily closing price for each stock and the target column.
- RowId: Unique ID of price records, the combination of Date and SecuritiesCode.
- Date: Trade date.
- SecuritiesCode: Local securities code.
- Open: First traded price on a day.
- High: Highest traded price on a day.
- Low: Lowest traded price on a day.
- Close: Last traded price on a day.
- Volume: Number of traded stocks on a day.
- AdjustmentFactor: Used to calculate theoretical price/volume when split/reverse-split happens (NOT including dividend/allotment of shares).
- ExpectedDividend: Expected dividend value for ex-right date. This value is recorded 2 business days before ex-dividend date.
- SupervisionFlag: Flag of securities under supervision and securities to be delisted, for more information, please see here.
- Target: Change ratio of adjusted closing price between t+2 and t+1 where t+0 is trade date.

**secondary_stock_prices.csv**: The core dataset contains on the 2,000 most commonly traded equities but many less liquid securities are also traded on the Tokyo market. This file contains data for those securities, which aren't scored but may be of interest for assessing the market as a whole.

**options.csv**: Data on the status of a variety of options based on the broader market. Many options include implicit predictions of the future price of the stock market and so may be of interest even though the options are not scored directly.

**trades.csv**: Aggregated summary of trading volumes from the previous business week.

**financials.csv**: Results from quarterly earnings reports.

**stock_list.csv**: Mapping between the SecuritiesCode and company names, plus general information about which industry the company is in.

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# local file
import functions

# Load data

Let's start with sample_submission to see what to expect in the competition

In [2]:
sample = pd.read_csv("jpx_tokyo_market_prediction/example_test_files/sample_submission.csv")
sample

,Date,SecuritiesCode,Rank
0,2021-12-06,1301,0
1,2021-12-06,1332,1
2,2021-12-06,1333,2
3,2021-12-06,1375,3
4,2021-12-06,1376,4
...,...,...,...
111995,2022-02-28,9990,1995
111996,2022-02-28,9991,1996
111997,2022-02-28,9993,1997
111998,2022-02-28,9994,1998


In [3]:
sample['Date'].value_counts()

2021-12-06    2000
2021-12-07    2000
2022-01-20    2000
2022-01-21    2000
2022-01-24    2000
2022-01-25    2000
2022-01-26    2000
2022-01-27    2000
2022-01-28    2000
2022-01-31    2000
2022-02-01    2000
2022-02-02    2000
2022-02-03    2000
2022-02-04    2000
2022-02-07    2000
2022-02-08    2000
2022-02-09    2000
2022-02-10    2000
2022-02-14    2000
2022-02-15    2000
2022-02-16    2000
2022-02-17    2000
2022-02-18    2000
2022-02-21    2000
2022-02-22    2000
2022-02-24    2000
2022-02-25    2000
2022-01-19    2000
2022-01-18    2000
2022-01-17    2000
2021-12-23    2000
2021-12-08    2000
2021-12-09    2000
2021-12-10    2000
2021-12-13    2000
2021-12-14    2000
2021-12-15    2000
2021-12-16    2000
2021-12-17    2000
2021-12-20    2000
2021-12-21    2000
2021-12-22    2000
2021-12-24    2000
2022-01-14    2000
2021-12-27    2000
2021-12-28    2000
2021-12-29    2000
2021-12-30    2000
2022-01-04    2000
2022-01-05    2000
2022-01-06    2000
2022-01-07    2000
2022-01-11  

In [4]:
sample.nunique()

Date                56
SecuritiesCode    2000
Rank              2000
dtype: int64

## Read all DataFrames

In [5]:
stock_list = pd.read_csv('jpx_tokyo_market_prediction/stock_list.csv')
financials = pd.read_csv('jpx_tokyo_market_prediction/train_files/financials.csv')
options = pd.read_csv('jpx_tokyo_market_prediction/train_files/options.csv')
stock_prices = pd.read_csv('jpx_tokyo_market_prediction/train_files/stock_prices.csv')
sec_stock_prices = pd.read_csv('jpx_tokyo_market_prediction/train_files/secondary_stock_prices.csv')
trades = pd.read_csv('jpx_tokyo_market_prediction/train_files/trades.csv')

C:\Users\Asus\AppData\Local\Temp\ipykernel_4408\3134442176.py:2: DtypeWarning: Columns (14,15,16,17,18,19,42,44) have mixed types. Specify dtype option on import or set low_memory=False.
  financials = pd.read_csv('jpx_tokyo_market_prediction/train_files/financials.csv')
C:\Users\Asus\AppData\Local\Temp\ipykernel_4408\3134442176.py:3: DtypeWarning: Columns (7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  options = pd.read_csv('jpx_tokyo_market_prediction/train_files/options.csv')


In [6]:
financials.head()

,DisclosureNumber,DateCode,Date,SecuritiesCode,DisclosedDate,DisclosedTime,DisclosedUnixTime,TypeOfDocument,CurrentPeriodEndDate,TypeOfCurrentPeriod,...,ForecastEarningsPerShare,ApplyingOfSpecificAccountingOfTheQuarterlyFinancialStatements,MaterialChangesInSubsidiaries,ChangesBasedOnRevisionsOfAccountingStandard,ChangesOtherThanOnesBasedOnRevisionsOfAccountingStandard,ChangesInAccountingEstimates,RetrospectiveRestatement,NumberOfIssuedAndOutstandingSharesAtTheEndOfFiscalYearIncludingTreasuryStock,NumberOfTreasuryStockAtTheEndOfFiscalYear,AverageNumberOfShares
0,2.016121e+13,20170104_2753,2017-01-04,2753.0,2017-01-04,07:30:00,1.483483e+09,3QFinancialStatements_Consolidated_JP,2016-12-31,3Q,...,319.76,NaN,False,True,False,False,False,6848800.0,－,6848800.0
1,2.017010e+13,20170104_3353,2017-01-04,3353.0,2017-01-04,15:00:00,1.483510e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,485.36,NaN,False,True,False,False,False,2035000.0,118917,1916083.0
2,2.016123e+13,20170104_4575,2017-01-04,4575.0,2017-01-04,12:00:00,1.483499e+09,ForecastRevision,2016-12-31,2Q,...,-93.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.017010e+13,20170105_2659,2017-01-05,2659.0,2017-01-05,15:00:00,1.483596e+09,3QFinancialStatements_Consolidated_JP,2016-11-30,3Q,...,285.05,NaN,False,True,False,False,False,31981654.0,18257,31963405.0
4,2.017011e+13,20170105_3050,2017-01-05,3050.0,2017-01-05,15:30:00,1.483598e+09,ForecastRevision,2017-02-28,FY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
financials.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92956 entries, 0 to 92955
Data columns (total 45 columns):
 #   Column                                                                        Non-Null Count  Dtype  
---  ------                                                                        --------------  -----  
 0   DisclosureNumber                                                              92954 non-null  float64
 1   DateCode                                                                      92954 non-null  object 
 2   Date                                                                          92956 non-null  object 
 3   SecuritiesCode                                                                92954 non-null  float64
 4   DisclosedDate                                                                 92954 non-null  object 
 5   DisclosedTime                                                                 92954 non-null  object 
 6   DisclosedUnixTime             

In [8]:
options.head()

,DateCode,Date,OptionsCode,WholeDayOpen,WholeDayHigh,WholeDayLow,WholeDayClose,NightSessionOpen,NightSessionHigh,NightSessionLow,...,Putcall,LastTradingDay,SpecialQuotationDay,SettlementPrice,TheoreticalPrice,BaseVolatility,ImpliedVolatility,InterestRate,DividendRate,Dividend
0,20170104_132010018,2017-01-04,132010018,650.0,650.0,480.0,480.0,0.0000,0.0000,0.0000,...,1,20170112,20170113,480.0,478.4587,17.4736,17.5865,0.0091,0.0,0.0
1,20170104_132010118,2017-01-04,132010118,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,1,20170112,20170113,575.0,571.1385,17.4736,16.5000,0.0091,0.0,0.0
2,20170104_132010218,2017-01-04,132010218,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,1,20170112,20170113,680.0,677.3710,17.4736,15.8644,0.0091,0.0,0.0
3,20170104_132010318,2017-01-04,132010318,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,1,20170112,20170113,795.0,791.0383,17.4736,15.2288,0.0091,0.0,0.0
4,20170104_132010518,2017-01-04,132010518,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0000,...,1,20170112,20170113,910.0,909.9947,17.4736,14.5932,0.0091,0.0,0.0


In [9]:
options.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3567694 entries, 0 to 3567693
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   DateCode             object 
 1   Date                 object 
 2   OptionsCode          int64  
 3   WholeDayOpen         float64
 4   WholeDayHigh         float64
 5   WholeDayLow          float64
 6   WholeDayClose        float64
 7   NightSessionOpen     object 
 8   NightSessionHigh     object 
 9   NightSessionLow      object 
 10  NightSessionClose    object 
 11  DaySessionOpen       float64
 12  DaySessionHigh       float64
 13  DaySessionLow        float64
 14  DaySessionClose      float64
 15  TradingVolume        int64  
 16  OpenInterest         int64  
 17  TradingValue         int64  
 18  ContractMonth        int64  
 19  StrikePrice          float64
 20  WholeDayVolume       int64  
 21  Putcall              int64  
 22  LastTradingDay       int64  
 23  SpecialQuotationDay  int64  
 24

In [10]:
sec_stock_prices.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1305,2017-01-04,1305,1594.0,1618.0,1594.0,1615.0,538190,1.0,NaN,False,-0.001855
1,20170104_1306,2017-01-04,1306,1575.0,1595.0,1573.0,1593.0,2494980,1.0,NaN,False,-0.000627
2,20170104_1308,2017-01-04,1308,1557.0,1580.0,1557.0,1578.0,526100,1.0,NaN,False,-0.001900
3,20170104_1309,2017-01-04,1309,28810.0,29000.0,28520.0,28780.0,403,1.0,NaN,False,0.005237
4,20170104_1311,2017-01-04,1311,717.0,735.0,717.0,734.0,5470,1.0,NaN,False,0.001359


In [11]:
sec_stock_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2384575 entries, 0 to 2384574
Data columns (total 12 columns):
 #   Column            Dtype  
---  ------            -----  
 0   RowId             object 
 1   Date              object 
 2   SecuritiesCode    int64  
 3   Open              float64
 4   High              float64
 5   Low               float64
 6   Close             float64
 7   Volume            int64  
 8   AdjustmentFactor  float64
 9   ExpectedDividend  float64
 10  SupervisionFlag   bool   
 11  Target            float64
dtypes: bool(1), float64(7), int64(2), object(2)
memory usage: 202.4+ MB


In [12]:
trades.head()

,Date,StartDate,EndDate,Section,TotalSales,TotalPurchases,TotalTotal,TotalBalance,ProprietarySales,ProprietaryPurchases,...,CityBKsRegionalBKsEtcTotal,CityBKsRegionalBKsEtcBalance,TrustBanksSales,TrustBanksPurchases,TrustBanksTotal,TrustBanksBalance,OtherFinancialInstitutionsSales,OtherFinancialInstitutionsPurchases,OtherFinancialInstitutionsTotal,OtherFinancialInstitutionsBalance
0,2017-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
trades.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1712 entries, 0 to 1711
Data columns (total 56 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Date                                 1712 non-null   object 
 1   StartDate                            765 non-null    object 
 2   EndDate                              765 non-null    object 
 3   Section                              765 non-null    object 
 4   TotalSales                           765 non-null    float64
 5   TotalPurchases                       765 non-null    float64
 6   TotalTotal                           765 non-null    float64
 7   TotalBalance                         765 non-null    float64
 8   ProprietarySales                     765 non-null    float64
 9   ProprietaryPurchases                 765 non-null    float64
 10  ProprietaryTotal                     765 non-null    float64
 11  ProprietaryBalance            

# E-D-A over J-P-X

## stock_list

In [14]:
stock_list.head()

,SecuritiesCode,EffectiveDate,Name,Section/Products,NewMarketSegment,33SectorCode,33SectorName,17SectorCode,17SectorName,NewIndexSeriesSizeCode,NewIndexSeriesSize,TradeDate,Close,IssuedShares,MarketCapitalization,Universe0
0,1301,20211230,"KYOKUYO CO.,LTD.",First Section (Domestic),Prime Market,50,"Fishery, Agriculture and Forestry",1,FOODS,7,TOPIX Small 2,20211230.0,3080.0,1.092828e+07,3.365911e+10,True
1,1305,20211230,Daiwa ETF-TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2097.0,3.634636e+09,7.621831e+12,False
2,1306,20211230,NEXT FUNDS TOPIX Exchange Traded Fund,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2073.5,7.917718e+09,1.641739e+13,False
3,1308,20211230,Nikko Exchange Traded Index Fund TOPIX,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,2053.0,3.736943e+09,7.671945e+12,False
4,1309,20211230,NEXT FUNDS ChinaAMC SSE50 Index Exchange Trade...,ETFs/ ETNs,NaN,-,-,-,-,-,-,20211230.0,44280.0,7.263200e+04,3.216145e+09,False


In [15]:
stock_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4417 entries, 0 to 4416
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SecuritiesCode          4417 non-null   int64  
 1   EffectiveDate           4417 non-null   int64  
 2   Name                    4417 non-null   object 
 3   Section/Products        4417 non-null   object 
 4   NewMarketSegment        3772 non-null   object 
 5   33SectorCode            4417 non-null   object 
 6   33SectorName            4417 non-null   object 
 7   17SectorCode            4417 non-null   object 
 8   17SectorName            4417 non-null   object 
 9   NewIndexSeriesSizeCode  4417 non-null   object 
 10  NewIndexSeriesSize      4417 non-null   object 
 11  TradeDate               4121 non-null   float64
 12  Close                   4121 non-null   float64
 13  IssuedShares            4121 non-null   float64
 14  MarketCapitalization    4121 non-null   

In [16]:
f"There are {len(stock_list['SecuritiesCode'].unique())} different stocks!"

'There are 4417 different stocks!'

In [17]:
sector_names_33 = stock_list[['33SectorName', '33SectorCode']].value_counts()
print(f'{len(sector_names_33)} different 33SectorName values')
sector_names_33

34 different 33SectorName values


33SectorName                                   33SectorCode
Information & Communication                    5250            551
Services                                       9050            521
-                                              -               500
Retail Trade                                   6100            359
Wholesale Trade                                6050            320
Electric Appliances                            3650            255
Machinery                                      3600            230
Chemicals                                      3200            218
Construction                                   2050            170
Real Estate                                    8050            143
Foods                                          3050            125
Other Products                                 3800            116
Transportation Equipment                       3700             95
Banks                                          7050             93
Me

In [18]:
sector_names_17 = stock_list[['17SectorName', '17SectorCode']].value_counts()
print(f'{len(sector_names_17)} different 17SectorName values')
print('Notice there are duplicates! FOODS, BANKS, etc.')
print('Need to merge them')
sector_names_17

34 different 17SectorName values
Notice there are duplicates! FOODS, BANKS, etc.
Need to merge them


17SectorName                                  17SectorCode
IT & SERVICES, OTHERS                         10              1155
-                                             -                500
RETAIL TRADE                                  14               339
COMMERCIAL & WHOLESALE TRADE                  13               311
CONSTRUCTION & MATERIALS                      3                303
ELECTRIC APPLIANCES & PRECISION INSTRUMENTS   9                293
RAW MATERIALS & CHEMICALS                     4                288
MACHINERY                                     8                228
REAL ESTATE                                   17               137
FOODS                                         1                136
TRANSPORTATION & LOGISTICS                    12               117
AUTOMOBILES & TRANSPORTATION EQUIPMENT        6                108
FINANCIALS （EX BANKS）                         16                88
BANKS                                         15                81
STE

In [26]:
# Distribution of above table (value_counts)
dist = stock_list.groupby('SecuritiesCode').size().reset_index(name='total')
stock_list_two = pd.merge(stock_list, dist, how='left', on=['SecuritiesCode'])
stock_list_two = stock_list_two.groupby(['17SectorName']).total.sum().reset_index(name='total')
fig = px.bar(x=stock_list_two["17SectorName"],
             y=stock_list_two['total'],
             color=stock_list_two["17SectorName"],
             color_continuous_scale="Emrld")
fig.update_xaxes(title="Assets")
fig.update_yaxes(title="Number of Rows")
fig.update_layout(showlegend=True,
                  title={
                      'text': 'Data Distribution ',  # Text of title
                      'y': 0.95,  # position of title in y-axis in numbers
                      'x': 0.5,  # position of title in x-axis in numbers
                      'xanchor': 'center',  # direction of position over title (x)
                      'yanchor': 'top'},  # direction of position over title (y)
                  template="plotly_white")
fig.show()

## stock_prices

In [20]:
stock_prices.describe()

,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,Target
count,2.332531e+06,2.324923e+06,2.324923e+06,2.324923e+06,2.324923e+06,2.332531e+06,2.332531e+06,18865.000000,2.332293e+06
mean,5.894835e+03,2.594511e+03,2.626540e+03,2.561227e+03,2.594023e+03,6.919366e+05,1.000508e+00,22.017730,4.450964e-04
std,2.404161e+03,3.577192e+03,3.619363e+03,3.533494e+03,3.576538e+03,3.911256e+06,6.773040e-02,29.882453,2.339879e-02
min,1.301000e+03,1.400000e+01,1.500000e+01,1.300000e+01,1.400000e+01,0.000000e+00,1.000000e-01,0.000000,-5.785414e-01
25%,3.891000e+03,1.022000e+03,1.035000e+03,1.009000e+03,1.022000e+03,3.030000e+04,1.000000e+00,5.000000,-1.049869e-02
50%,6.238000e+03,1.812000e+03,1.834000e+03,1.790000e+03,1.811000e+03,1.071000e+05,1.000000e+00,15.000000,0.000000e+00
75%,7.965000e+03,3.030000e+03,3.070000e+03,2.995000e+03,3.030000e+03,4.021000e+05,1.000000e+00,30.000000,1.053159e-02
max,9.997000e+03,1.099500e+05,1.105000e+05,1.072000e+05,1.095500e+05,6.436540e+08,2.000000e+01,1070.000000,1.119512e+00


In [21]:
stock_prices.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026


In [22]:
stock_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2332531 entries, 0 to 2332530
Data columns (total 12 columns):
 #   Column            Dtype  
---  ------            -----  
 0   RowId             object 
 1   Date              object 
 2   SecuritiesCode    int64  
 3   Open              float64
 4   High              float64
 5   Low               float64
 6   Close             float64
 7   Volume            int64  
 8   AdjustmentFactor  float64
 9   ExpectedDividend  float64
 10  SupervisionFlag   bool   
 11  Target            float64
dtypes: bool(1), float64(7), int64(2), object(2)
memory usage: 198.0+ MB


### See how a specific stock behaves on each of its trading days:

In [23]:
temp_df = stock_prices[stock_prices["SecuritiesCode"] == 1301].reset_index(drop=True)
functions.calculate_target_as_rate(temp_df)
temp_df.head()

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,Close_shift1,Close_shift2,rate
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730,2738.0,2740.0,0.000730
1,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920,2740.0,2748.0,0.002920
2,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092,2748.0,2745.0,-0.001092
3,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100,2745.0,2731.0,-0.005100
4,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295,2731.0,2722.0,-0.003295


### On a specific day - rank all stocks by "Target". This is when we choose the top and botttom 200 stocks for buying/selling on the next trading day

In [24]:
tmpdf2 = stock_prices[stock_prices["Date"] == "2021-12-02"].reset_index(drop=True)
tmpdf2["rank"] = tmpdf2["Target"].rank(ascending=False, method="first") - 1
tmpdf2 = tmpdf2.sort_values("rank").reset_index(drop=True)
tmpdf2

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target,rank
0,20211202_4699,2021-12-02,4699,2037.0,2149.0,2037.0,2143.0,31000,1.0,NaN,False,0.231481,0.0
1,20211202_4488,2021-12-02,4488,5900.0,5990.0,5690.0,5700.0,57100,1.0,NaN,False,0.149254,1.0
2,20211202_8057,2021-12-02,8057,4710.0,4820.0,4615.0,4640.0,29800,1.0,NaN,False,0.123110,2.0
3,20211202_9632,2021-12-02,9632,8110.0,8150.0,8000.0,8030.0,3900,1.0,NaN,False,0.107011,3.0
4,20211202_6958,2021-12-02,6958,510.0,525.0,502.0,517.0,639200,1.0,NaN,False,0.103113,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,20211202_4483,2021-12-02,4483,8200.0,8590.0,8180.0,8410.0,320300,1.0,NaN,False,-0.085480,1995.0
1996,20211202_6879,2021-12-02,6879,731.0,745.0,702.0,707.0,735500,1.0,NaN,False,-0.087248,1996.0
1997,20211202_4168,2021-12-02,4168,4280.0,4390.0,4200.0,4335.0,72600,1.0,NaN,False,-0.094595,1997.0
1998,20211202_3994,2021-12-02,3994,7490.0,7500.0,7060.0,7070.0,735000,1.0,NaN,False,-0.094980,1998.0


### Keep stocks with data during <u>all trading days</u> and remove others that are not traded every day. These remaining stocks are the ones we are going to analyze and "play" with from now on.

In [28]:
traded_stock_per_date = stock_prices['Date'].value_counts()
print(f'We have {len(traded_stock_per_date)} days of data with ~{round(np.mean(traded_stock_per_date.values))} rows per day!\n'
      f'Where {traded_stock_per_date.values.min()} is the least for a certain day and {traded_stock_per_date.values.max()} is the most')
traded_stock_per_date

We have 1202 days of data with ~1941 rows per day!
Where 1865 is the least for a certain day and 2000 is the most


2021-12-03    2000
2021-04-08    2000
2021-04-27    2000
2021-04-26    2000
2021-04-23    2000
              ... 
2017-02-08    1865
2017-02-07    1865
2017-02-06    1865
2017-02-03    1865
2017-01-04    1865
Name: Date, Length: 1202, dtype: int64

In [30]:
stocks_by_descending_trade_count = stock_prices['SecuritiesCode'].value_counts()
stocks_by_descending_trade_count

1301    1202
7282    1202
7414    1202
7412    1202
7408    1202
        ... 
4167     235
7358     234
4168     233
7342     233
4169     232
Name: SecuritiesCode, Length: 2000, dtype: int64

In [35]:
def get_stocks_traded_every_day(stock_prices_df):
    """Return stocks that are traded in every trading day in data and remove others. These remaining stocks are the ones we are going to analyze and "play" with from now on"""
    traded_stock_per_date = stock_prices_df['Date'].value_counts()
    stocks_by_descending_trade_count = stock_prices_df['SecuritiesCode'].value_counts()
    return stocks_by_descending_trade_count[stocks_by_descending_trade_count == len(stock_prices_date)].index.values

daily_traded_stocks = get_stocks_traded_every_day(stock_prices)
daily_traded_stocks

array([1301, 7282, 7414, ..., 4565, 4369, 4559], dtype=int64)

# Next steps
1. Decide which features are relevant for training (i.e. volume_last_week, m_cap_last_month, NetSales, etc.)
2. Create a training set with the selected features per stock (in daily_traded_stocks) per day.
    The training set should have the following structure:
        X:
        [
            [volume_last_week, m_cap_last_month, etc.]
            ...
            [volume_last_week, m_cap_last_month, etc.]
        ]

        Y: Target value per row

3. Now we need to decide on a model for predicting Y.
4. Predict the target column, Y.
    Note, it's the return from buying a stock the **next day and selling the day after that** so should we assign the target of row i at i+2 ???
5. Calculate loss - decide on MSE/MAE/etc.
6. Discuss improvements of prediction, model, and code quality (runtime for example)

In [37]:
# Volume - number of traded stocks on a day